# Join Service Requests Data to Geo Hex Data


In [1]:
# Imports
import requests
import gzip
import shutil

from tqdm import tqdm
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
def download_file(url, filename):
    try:
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get('content-length', 0))
        response.raise_for_status()  # Raises an HTTPError for bad responses
        
        with open(filename, "wb") as file, tqdm(
            desc=filename,
            total=total_size,
            unit='B',
            unit_scale=True,
            unit_divisor=1024,
        ) as pbar:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
                pbar.update(len(chunk))
        
        print(f"Downloaded: {filename}")
        return True
    
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")
        return False
    
def uncompress_file(compressed_filename, uncompressed_filename):
    with gzip.open(compressed_filename, 'rb') as f_in:
        with open(uncompressed_filename, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
    print(f"Uncompressed: {compressed_filename} to {uncompressed_filename}")
    return True

In [12]:
DATA_URL = "https://cct-ds-code-challenge-input-data.s3.af-south-1.amazonaws.com/sr.csv.gz"
COMPRESSED_FILE_NAME = "sr.csv.gz"
UNCOMPRESSED_FILE_NAME = "sr.csv"

download_file(DATA_URL, COMPRESSED_FILE_NAME)

uncompress_file(COMPRESSED_FILE_NAME, UNCOMPRESSED_FILE_NAME)

sr.csv.gz: 100%|██████████| 35.5M/35.5M [00:04<00:00, 7.74MB/s]


Downloaded: sr.csv.gz
Uncompressed: sr.csv.gz to sr.csv


True

In [4]:
DATA_URL = "https://cct-ds-code-challenge-input-data.s3.af-south-1.amazonaws.com/sr_hex.csv.gz"
COMPRESSED_FILE_NAME = "sr_hex.csv.gz"
UNCOMPRESSED_FILE_NAME = "sr_hex.csv"

download_file(DATA_URL, COMPRESSED_FILE_NAME)

uncompress_file(COMPRESSED_FILE_NAME, UNCOMPRESSED_FILE_NAME)

sr_hex.csv.gz: 100%|██████████| 35.0M/35.0M [00:04<00:00, 7.55MB/s]


Downloaded: sr_hex.csv.gz
Uncompressed: sr_hex.csv.gz to sr_hex.csv


True

In [5]:
DATA_URL = "https://cct-ds-code-challenge-input-data.s3.af-south-1.amazonaws.com/city-hex-polygons-8.geojson"
FILE_NAME = "city-hex-polygons-8.geojson"

download_file(DATA_URL, FILE_NAME)

city-hex-polygons-8.geojson: 100%|██████████| 1.88M/1.88M [00:00<00:00, 3.67MB/s]

Downloaded: city-hex-polygons-8.geojson


True

# Load data
We will load the SR data into a dataframe and the geojson data into a geodataframe.

In [4]:
sr_df = pd.read_csv("sr.csv")
sr_df.info()
sr_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941634 entries, 0 to 941633
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Unnamed: 0            941634 non-null  int64  
 1   notification_number   941634 non-null  int64  
 2   reference_number      592920 non-null  float64
 3   creation_timestamp    941634 non-null  object 
 4   completion_timestamp  929442 non-null  object 
 5   directorate           932199 non-null  object 
 6   department            932180 non-null  object 
 7   branch                913233 non-null  object 
 8   section               848509 non-null  object 
 9   code_group            941634 non-null  object 
 10  code                  941634 non-null  object 
 11  cause_code_group      131117 non-null  object 
 12  cause_code            129669 non-null  object 
 13  official_suburb       729221 non-null  object 
 14  latitude              729270 non-null  float64
 15  

,Unnamed: 0,notification_number,reference_number,creation_timestamp,completion_timestamp,directorate,department,branch,section,code_group,code,cause_code_group,cause_code,official_suburb,latitude,longitude
0,0,400583534,9.109492e+09,2020-10-07 06:55:18+02:00,2020-10-08 15:36:35+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area Central,District: Blaauwberg,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Wear and tear,MONTAGUE GARDENS,-33.872839,18.522488
1,1,400555043,9.108995e+09,2020-07-09 16:08:13+02:00,2020-07-14 14:27:01+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,SOMERSET WEST,-34.078916,18.848940
2,2,400589145,9.109614e+09,2020-10-27 10:21:59+02:00,2020-10-28 17:48:15+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,STRAND,-34.102242,18.821116
3,3,400538915,9.108601e+09,2020-03-19 06:36:06+02:00,2021-03-29 20:34:19+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area North,District : Bellville,TD Customer complaint groups,Paint Markings Lines&Signs,Road Markings,Wear and tear,RAVENSMEAD,-33.920019,18.607209
4,4,400568554,NaN,2020-08-25 09:48:42+02:00,2020-08-31 08:41:13+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area South,District : Athlone,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Surfacing failure,CLAREMONT,-33.987400,18.453760


In [5]:
# Read your file into a GeoDataFrame
gdf = gpd.read_file("city-hex-polygons-8.geojson")

# Inspect
print(gdf.head())
print(gdf.crs)          # Coordinate reference system
print(gdf.geometry.type.value_counts())  # What kinds of geometries?

             index  centroid_lat  centroid_lon  \
0  88ad361801fffff    -33.859427     18.677843   
1  88ad361803fffff    -33.855696     18.668766   
2  88ad361805fffff    -33.855263     18.685959   
3  88ad361807fffff    -33.851532     18.676881   
4  88ad361809fffff    -33.867322     18.678806   

                                            geometry  
0  POLYGON ((18.68119 -33.8633, 18.68357 -33.8592...  
1  POLYGON ((18.67211 -33.85957, 18.6745 -33.8555...  
2  POLYGON ((18.68931 -33.85914, 18.69169 -33.855...  
3  POLYGON ((18.68023 -33.85541, 18.68261 -33.851...  
4  POLYGON ((18.68215 -33.8712, 18.68454 -33.8671...  
EPSG:4326
Polygon    3832
Name: count, dtype: int64


# Turn SR Data into GeoDataFrame of Points
We will convert the SR data into a GeoDataFrame of points so we can do a spatial join.

In [6]:
points = gpd.GeoDataFrame(
    sr_df,
    geometry = [Point(xy) for xy in zip(sr_df["longitude"], sr_df["latitude"])],
    crs = "EPSG:4326"
)

points.head()

,Unnamed: 0,notification_number,reference_number,creation_timestamp,completion_timestamp,directorate,department,branch,section,code_group,code,cause_code_group,cause_code,official_suburb,latitude,longitude,geometry
0,0,400583534,9.109492e+09,2020-10-07 06:55:18+02:00,2020-10-08 15:36:35+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area Central,District: Blaauwberg,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Wear and tear,MONTAGUE GARDENS,-33.872839,18.522488,POINT (18.52249 -33.87284)
1,1,400555043,9.108995e+09,2020-07-09 16:08:13+02:00,2020-07-14 14:27:01+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,SOMERSET WEST,-34.078916,18.848940,POINT (18.84894 -34.07892)
2,2,400589145,9.109614e+09,2020-10-27 10:21:59+02:00,2020-10-28 17:48:15+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,STRAND,-34.102242,18.821116,POINT (18.82112 -34.10224)
3,3,400538915,9.108601e+09,2020-03-19 06:36:06+02:00,2021-03-29 20:34:19+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area North,District : Bellville,TD Customer complaint groups,Paint Markings Lines&Signs,Road Markings,Wear and tear,RAVENSMEAD,-33.920019,18.607209,POINT (18.60721 -33.92002)
4,4,400568554,NaN,2020-08-25 09:48:42+02:00,2020-08-31 08:41:13+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area South,District : Athlone,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Surfacing failure,CLAREMONT,-33.987400,18.453760,POINT (18.45376 -33.9874)


In [7]:
# Spatial join

joined = gpd.sjoin(
    points,
    gdf[["geometry","index"]],  # keep only the hex index
    how="left",
    predicate="within"
)

joined.head()

,Unnamed: 0,notification_number,reference_number,creation_timestamp,completion_timestamp,directorate,department,branch,section,code_group,code,cause_code_group,cause_code,official_suburb,latitude,longitude,geometry,index_right0,index
0,0,400583534,9.109492e+09,2020-10-07 06:55:18+02:00,2020-10-08 15:36:35+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area Central,District: Blaauwberg,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Wear and tear,MONTAGUE GARDENS,-33.872839,18.522488,POINT (18.52249 -33.87284),1047.0,88ad360225fffff
1,1,400555043,9.108995e+09,2020-07-09 16:08:13+02:00,2020-07-14 14:27:01+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,SOMERSET WEST,-34.078916,18.848940,POINT (18.84894 -34.07892),3055.0,88ad36d5e1fffff
2,2,400589145,9.109614e+09,2020-10-27 10:21:59+02:00,2020-10-28 17:48:15+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,STRAND,-34.102242,18.821116,POINT (18.82112 -34.10224),2946.0,88ad36d437fffff
3,3,400538915,9.108601e+09,2020-03-19 06:36:06+02:00,2021-03-29 20:34:19+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area North,District : Bellville,TD Customer complaint groups,Paint Markings Lines&Signs,Road Markings,Wear and tear,RAVENSMEAD,-33.920019,18.607209,POINT (18.60721 -33.92002),1247.0,88ad361133fffff
4,4,400568554,NaN,2020-08-25 09:48:42+02:00,2020-08-31 08:41:13+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area South,District : Athlone,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Surfacing failure,CLAREMONT,-33.987400,18.453760,POINT (18.45376 -33.9874),2530.0,88ad361709fffff


In [ ]:
# drop unused columns
joined.drop(columns=["index_right0", "geometry", "Unnamed: 0"], inplace=True)

# rename index to hex_id
joined.rename(columns={"index": "h3_level8_index"}, inplace=True)

# replace nan with "0" in h3_level8_index
joined["h3_level8_index"] = joined["h3_level8_index"].fillna("0")


In [9]:
joined.head()

,notification_number,reference_number,creation_timestamp,completion_timestamp,directorate,department,branch,section,code_group,code,cause_code_group,cause_code,official_suburb,latitude,longitude,h3_level8_index
0,400583534,9.109492e+09,2020-10-07 06:55:18+02:00,2020-10-08 15:36:35+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area Central,District: Blaauwberg,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Wear and tear,MONTAGUE GARDENS,-33.872839,18.522488,88ad360225fffff
1,400555043,9.108995e+09,2020-07-09 16:08:13+02:00,2020-07-14 14:27:01+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,SOMERSET WEST,-34.078916,18.848940,88ad36d5e1fffff
2,400589145,9.109614e+09,2020-10-27 10:21:59+02:00,2020-10-28 17:48:15+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area East,District : Somerset West,TD Customer complaint groups,Manhole Cover/Gully Grid,Road (RCL),Vandalism,STRAND,-34.102242,18.821116,88ad36d437fffff
3,400538915,9.108601e+09,2020-03-19 06:36:06+02:00,2021-03-29 20:34:19+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area North,District : Bellville,TD Customer complaint groups,Paint Markings Lines&Signs,Road Markings,Wear and tear,RAVENSMEAD,-33.920019,18.607209,88ad361133fffff
4,400568554,NaN,2020-08-25 09:48:42+02:00,2020-08-31 08:41:13+02:00,URBAN MOBILITY,Roads Infrastructure Management,RIM Area South,District : Athlone,TD Customer complaint groups,Pothole&Defect Road Foot Bic Way/Kerbs,Road (RCL),Surfacing failure,CLAREMONT,-33.987400,18.453760,88ad361709fffff


### Validation

In [17]:
sr_hex = pd.read_csv("sr_hex.csv")

print((sr_hex["notification_number"] == joined["notification_number"]).all())

print(f"{(sr_hex['h3_level8_index'] == joined['h3_level8_index']).sum()} of {sr_hex['notification_number'].count()} are the same.")
print(f"{sr_hex['notification_number'].count() - (sr_hex['h3_level8_index'] == joined['h3_level8_index']).sum()} are different.")

True
941605 of 941634 are the same.
29 are different.


We see that the vast majority are assigned the same as in the validation data. Only 29 are different for some reasone. Unfortunatly I could not figure out why, it may be because the validation set is wrong..